In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from skopt.space import Real, Integer
from skopt.utils import use_named_args
from skopt import gp_minimize
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import LassoCV
import xgboost as xgb
import lightgbm as lgb
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras import backend as K
from sklearn.ensemble import RandomForestRegressor

Using TensorFlow backend.


Predicciones 4 df juntos

In [3]:
data = pd.read_pickle('nuevos_modelos/ventana5.pkl')
target = pd.read_csv('data/target_competencia_ids.csv')

In [4]:
target['ref_hash'] = target['ref_hash'].apply(lambda x: int(str(x)[:-3]))
target = target.drop_duplicates('ref_hash')

In [6]:
#Predicciones auctions:
lgb_model = pickle.load(open('nuevos_modelos/lgb1_all_auctions', 'rb'))

data = pd.merge(data, target, on='ref_hash', how='right')
data['last_auction_sec_to_end'] = data['last_auction_sec_to_end'].fillna(3600*24*3)
data = data.fillna(0)

pred_data_auctions = model_data_auctions.predict(data.drop(columns=['ref_hash']))
predictions_auctions = pd.DataFrame({'ref_hash': data['ref_hash'],'obj': pred_data_auctions})

predictions_auctions['ref_hash'] = predictions_auctions['ref_hash'].apply(lambda x: str(x) + '_st')

In [13]:
#Predicciones installs:
model_data_auctions = lgb.Booster(model_file='modelos/lightgbm_all_installs')

pred_data_installs = model_data_auctions.predict(data.drop(columns=['ref_hash']))
predictions_installs = pd.DataFrame({'ref_hash': data['ref_hash'],'obj': pred_data_installs})

predictions_installs['ref_hash'] = predictions_installs['ref_hash'].apply(lambda x: str(x) + '_sc')

In [16]:
predictions = pd.concat([predictions_auctions, predictions_installs])

In [20]:
predictions.to_csv('predicciones/predicciones11.csv', index=False)

In [22]:
predictions_installs_2 = pd.read_csv('predicciones/installs_preds3.csv')

In [24]:
predictions2 = pd.merge(predictions, predictions_installs_2, on='ref_hash', how='left')

In [31]:
predictions2 = predictions2.fillna(0)

In [32]:
predictions2['obj'] = predictions2.apply(lambda x: x['obj_x'] if x['obj_y'] == 0 else x['obj_x']*0.25+x['obj_y']*0.75, axis=1)

In [36]:
predictions2['obj'] = predictions2['obj'].apply(lambda x: 0 if x<0 else x)

In [38]:
predictions2[['ref_hash', 'obj']].to_csv('predicciones/predicciones12.csv', index=False)

Predicciones 4 df separados

In [3]:
auctions = pd.read_pickle('data/model/model_auc_w5.pkl')
clicks = pd.read_pickle('data/model/model_cli_w5.pkl')
events = pd.read_pickle('data/model/model_eve_w5.pkl')
installs = pd.read_pickle('data/model/model_ins_w5.pkl')
target = pd.read_csv('data/target_competencia_ids.csv')

In [4]:
target['ref_hash'] = target['ref_hash'].apply(lambda x: int(str(x)[:-3]))
target = target.drop_duplicates('ref_hash')

In [5]:
auctions = pd.merge(auctions, target, on='ref_hash', how='right')
auctions['last_auction_sec_to_end'] = auctions['last_auction_sec_to_end'].fillna(3600*24*3)
auctions = auctions.fillna(0)

clicks = pd.merge(clicks, target, on='ref_hash', how='inner')

In [6]:
#Predicciones auctions:
model_auctions_auctions = lgb.Booster(model_file='modelos/lightgbm_auctions_auctions')
model_clicks_auctions = lgb.Booster(model_file='modelos/lightgbm_clicks_auctions')

pred_auctions_auctions = model_auctions_auctions.predict(auctions.drop(columns=['ref_hash']))
pred_auctions_auctions = pd.DataFrame({'ref_hash': auctions['ref_hash'],'obj_auc': pred_auctions_auctions})
pred_clicks_auctions = model_clicks_auctions.predict(clicks.drop(columns=['ref_hash']))
pred_clicks_auctions = pd.DataFrame({'ref_hash': clicks['ref_hash'],'obj_cli': pred_clicks_auctions})

predictions_auctions = pd.merge(pred_auctions_auctions, pred_clicks_auctions, on='ref_hash', how='outer')

predictions_auctions['obj'] = predictions_auctions.apply(lambda x: x['obj_auc'] if np.isnan(x['obj_cli']) else x['obj_auc']*0.5 + x['obj_cli']*0.5, axis=1)
predictions_auctions['obj'] = predictions_auctions['obj'].apply(lambda x: 0 if x<0 else x)
predictions_auctions.drop(columns=['obj_auc', 'obj_cli'], inplace=True)
predictions_auctions['ref_hash'] = predictions_auctions['ref_hash'].apply(lambda x: str(x) + '_st')

In [7]:
#Predicciones installs:
model_auctions_installs = lgb.Booster(model_file='modelos/lightgbm_auctions_installs')
model_events_installs = lgb.Booster(model_file='modelos/lightgbm_events_installs')
model_clicks_installs = lgb.Booster(model_file='modelos/lightgbm_clicks_installs')
model_installs_installs = lgb.Booster(model_file='modelos/lightgbm_installs_installs')

events = pd.merge(events, target, on='ref_hash', how='right')
events = events.fillna(0)

installs = pd.merge(installs, target, on='ref_hash', how='right')
installs = installs.fillna(0)

pred_auctions_installs = model_auctions_installs.predict(auctions.drop(columns=['ref_hash']))
pred_auctions_installs = pd.DataFrame({'ref_hash': auctions['ref_hash'],'obj_auc': pred_auctions_installs})
pred_clicks_installs = model_clicks_installs.predict(clicks.drop(columns=['ref_hash']))
pred_clicks_installs = pd.DataFrame({'ref_hash': clicks['ref_hash'],'obj_cli': pred_clicks_installs})
pred_events_installs = model_events_installs.predict(events.drop(columns=['ref_hash']))
pred_events_installs = pd.DataFrame({'ref_hash': events['ref_hash'],'obj_eve': pred_events_installs})
pred_installs_installs = model_installs_installs.predict(installs.drop(columns=['ref_hash']))
pred_installs_installs = pd.DataFrame({'ref_hash': installs['ref_hash'],'obj_ins': pred_installs_installs})

predictions_installs = pd.merge(pred_auctions_installs, pred_clicks_installs, on='ref_hash', how='outer')
predictions_installs = pd.merge(predictions_installs, pred_events_installs, on='ref_hash', how='outer')
predictions_installs = pd.merge(predictions_installs, pred_installs_installs, on='ref_hash', how='outer')

predictions_installs['obj'] = predictions_installs.apply(lambda x: x['obj_auc']*(1/3) + x['obj_eve']*(1/3) + x['obj_ins']*(1/3) if np.isnan(x['obj_cli']) else 
                                                         x['obj_auc']*0.25 + x['obj_ins']*0.25 + x['obj_eve']*0.25 + x['obj_cli']*0.25, axis=1)
predictions_installs['obj'] = predictions_installs['obj'].apply(lambda x: 0 if x<0 else x)
predictions_installs.drop(columns=['obj_auc', 'obj_cli', 'obj_eve', 'obj_ins'], inplace=True)
predictions_installs['ref_hash'] = predictions_installs['ref_hash'].apply(lambda x: str(x) + '_sc')

In [12]:
predictions_installs.sort_values('obj')

,ref_hash,obj
35,1999681247947819001_sc,206810.692606
712,2021644029481158151_sc,213572.622886
2775,4693183033162053177_sc,216913.050349
2101,7921421656863068740_sc,217083.345006
3425,1677397424183807063_sc,217413.358379
216,529957280949946515_sc,219093.418821
1386,4327376573650330232_sc,220246.159768
2214,6736512068450951714_sc,220312.511145
2470,4079581877269496973_sc,220970.785959
822,5352962120514576625_sc,221413.084034


In [13]:
predictions = pd.concat([predictions_installs, predictions_auctions])

In [26]:
predictions.to_csv('predicciones/predicciones10.csv', index=False)

In [27]:
predictions.sort_values('ref_hash').head(10)

,ref_hash,obj
2564,1000169251625791246_sc,240583.781932
2564,1000169251625791246_st,41590.413862
28,1000395625957344683_sc,242295.975188
28,1000395625957344683_st,40909.319396
82,1003027494996471685_sc,245929.026701
82,1003027494996471685_st,85632.800225
1581,1006670001679961544_sc,241310.829060
1581,1006670001679961544_st,98524.929235
403,1007573308966476713_sc,241999.654501
403,1007573308966476713_st,59637.176179


In [24]:
predictions_old = pd.read_csv('predicciones/predicciones7.csv')

In [25]:
predictions_old.sort_values('ref_hash').head(10)

,ref_hash,obj
0,1000169251625791246_sc,80796.780000
1,1000169251625791246_st,18856.525697
2,1000395625957344683_sc,101719.352652
3,1000395625957344683_st,21424.327136
4,1003027494996471685_sc,418.346150
5,1003027494996471685_st,53520.618984
6,1006670001679961544_sc,532286.212410
7,1006670001679961544_st,52050.308920
8,1007573308966476713_sc,99440.314369
9,1007573308966476713_st,28974.876761


In [28]:
predictions_old.mean()

obj    85409.651787
dtype: float64